In [78]:
import numpy as np
import mygrad as mg

In [79]:
class sigmoid():
    @staticmethod
    def activation(x):
        y = 1 / (1 + np.exp(-x))
        return y
    @staticmethod
    def prime(y):
        f = sigmoid.activation(y)
        return (f * (1 - f))

In [80]:
class CategoricalCrossEntropy():
    def __init__(self, a, y_true,_m):
        print(a.shape, np.sum(a,axis=-1,keepdims=True))
        self.p = a / np.sum(a,axis=-1,keepdims=True)
        self.y = y_true
        self.m = _m
    def metrics(self): # -y.log(p)
        m = self.p.shape[0]
        return (-1 / m) * np.sum((self.y * np.log(self.p+1e-8)))
    def forward(self): # -y.log(p)
        m = self.p.shape[0]
        return np.sum((-self.y * np.log(self.p+1e-8)))
    def backward(self):
    # -y/p
        return (-self.y/(self.p+1e-8))

In [89]:
class MSE():
    def __init__(self, p, y, _m):
        self.p = p
        self.y = y
        self.m = _m
    def forward(self):
        return (np.square(self.y - self.p))
    def backward(self):
        return (-2*(self.y - self.p))
    def metrics(self):
        m = self.p.shape[0]
        return (1 / m) * (np.sum(np.square(self.y - self.p)))

In [90]:
class softmax:
    @staticmethod
    def activation(x):
        # e_x = np.exp(x).reshape(-1,1)
        e_x = np.exp(x - np.max(x,axis=-1).reshape(-1,1))
        return e_x / e_x.sum(axis=-1).reshape(-1,1)
    @staticmethod
    def primeP(y): #marche pas
        n = y.shape[1]
        print(n,y.shape)
        i = np.identity(n)
        un = np.ones((n,y.shape[1]))
        SM = y.reshape((-1,1))
        print(SM.shape)
        jac = np.diagflat(y) - np.dot(SM, SM.T)
        j = np.dot((i* jac),un)
        # x = np.dot(jac,y.T)
        return j.T
    @staticmethod
    def prime(y):
        return(y * (1-y))

In [91]:
class BinaryCrossEntropy:
    def __init__(self, p, y,_m):
        self.p = p
        self.y = y
        self.m = _m
    def metrics(self):
        m = self.p.shape[0]
        return (-1 / m) * (np.sum((self.y * np.log(self.p + 1e-8)) + ((1 - self.y) * (np.log(1 - self.p + 1e-8)))))

    def forward(self):
    # -(y.log(p) + (1-y).log(1-p))
        return (-self.y * np.log(self.p + 1e-8)) + ((1 - self.y) * (np.log(1 - self.p + 1e-8)))

    def backward(self):
    # -(y/p + (1-y)/(1-p))
        return (-self.y/(self.p+1e-8) + (1 - self.y)/(1 - self.p))


In [92]:
np.random.seed(128)
x = np.random.randn(2,2)
y = np.array([[1,0,0],[0,1,0]])
w = np.random.randn(2,3)
b = np.zeros((3))
z = np.dot(x,w) + b
a = sigmoid.activation(z)
# a = sigmoid.activation(z)
print(a)
dL = a - y
dL

[[0.40204994 0.28027011 0.52809671]
 [0.67968693 0.85353514 0.46513931]]


array([[-0.59795006,  0.28027011,  0.52809671],
       [ 0.67968693, -0.14646486,  0.46513931]])

In [93]:
LossFct = MSE(a,y,0)
L = LossFct.forward()
L

array([[0.35754427, 0.07855133, 0.27888614],
       [0.46197432, 0.02145196, 0.21635458]])

In [94]:
dL = LossFct.backward()
dL

array([[-0.59795006,  0.28027011,  0.52809671],
       [ 0.67968693, -0.14646486,  0.46513931]])

In [95]:
# da_dz = sigmoid.prime(a)
# da_dz = softmax.prime(a)
da_dz = sigmoid.prime(a)

print(da_dz)

[[0.24016331 0.24515411 0.23334891]
 [0.22321252 0.20947501 0.2369509 ]]


In [96]:
print(a-y)
print(L)
print(dL)
print(da_dz)
print((dL*da_dz))

[[-0.59795006  0.28027011  0.52809671]
 [ 0.67968693 -0.14646486  0.46513931]]
[[0.35754427 0.07855133 0.27888614]
 [0.46197432 0.02145196 0.21635458]]
[[-0.59795006  0.28027011  0.52809671]
 [ 0.67968693 -0.14646486  0.46513931]]
[[0.24016331 0.24515411 0.23334891]
 [0.22321252 0.20947501 0.2369509 ]]
[[-0.14360567  0.06870937  0.12323079]
 [ 0.15171463 -0.03068073  0.11021518]]


In [97]:
dL = LossFct.backward()
print(softmax.prime(L))
dL_da = np.dot(dL,softmax.prime(L))
dL_da

[[0.22970637 0.07238102 0.20110866]
 [0.24855405 0.02099177 0.16954527]]


ValueError: shapes (2,3) and (2,3) not aligned: 3 (dim 1) != 2 (dim 0)

In [60]:
i = np.identity(3)
un = np.ones((3,1))
j = np.dot((i* da_dz),un)
j

ValueError: operands could not be broadcast together with shapes (3,3) (2,3) 